# Study Design: Customer Segmentation

This notebook defines the **2-Track Study Design** for customer segmentation research.

## Table of Contents

1. [Overview & Framework](#1.-Overview-&-Framework)
2. [Track Relationships & Practical Usage](#2.-Track-Relationships-&-Practical-Usage)
3. [Track 1: Customer Understanding](#3.-Track-1:-Customer-Understanding)
   - 3.1 Methods
   - 3.2 Operational Definitions
   - 3.3 Features
   - 3.4 Evaluation
4. [Track 2: Causal Targeting](#4.-Track-2:-Causal-Targeting)
   - 4.1 Methods
   - 4.2 Operational Definitions (3 Scenarios)
   - 4.3 Causal Graph
   - 4.4 Features
   - 4.5 Evaluation
5. [Period & Validation Design](#5.-Period-&-Validation-Design)
6. [Implementation Plan](#6.-Implementation-Plan)
7. [Cohort Definition & Comparison](#7.-Cohort-Definition-&-Comparison)

---

## 1. Overview & Framework

### 1.1 Framework Structure

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         2-TRACK STUDY DESIGN                                │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRACK 1: CUSTOMER UNDERSTANDING (Descriptive)                              │
│  ═══════════════════════════════════════════════                            │
│                                                                             │
│  Step 1.1: Exploratory                                                      │
│    - Factor Analysis (NMF) → Discover latent dimensions                     │
│    - Clustering → Derive base segments                                      │
│                                                                             │
│  Step 1.2: Value × Need Integration                                         │
│    - Value layer: CLV, Engagement                                           │
│    - Need layer: Behavior, Category preference                              │
│    - Integration: Value-Need Matrix or 2D Segmentation                      │
│                                                                             │
│  Output: "To whom (Value) should we offer what (Need)?"                     │
│                                                                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRACK 2: CAUSAL TARGETING                                                  │
│  ═════════════════════════                                                  │
│                                                                             │
│  Step 2.1: HTE Analysis                                                     │
│    - Campaign effect heterogeneity                                          │
│    - Covariates: Track 1 segments + raw features                            │
│                                                                             │
│  Step 2.2: Optimal Policy                                                   │
│    - Targeting rules                                                        │
│    - Policy value estimation                                                │
│                                                                             │
│  Output: "How do we optimize targeting?"                                    │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 1.2 Design Rationale

The 2-Track approach separates:
- **Descriptive analysis** (understanding customers) from
- **Causal analysis** (optimizing interventions)

This separation enables:
1. Different teams to work on appropriate tracks based on expertise
2. Track 1 to provide immediate marketing value
3. Track 2 to build on Track 1 segments for causal insights

---

## 2. Track Relationships & Practical Usage

### 2.1 Track Relationships

```
Step 1.1 (Exploratory) ──────────────────────────┐
    │                                            │
    │ (use if useful, otherwise independent)     │ (selective factor usage)
    ↓                                            ↓
Step 1.2 (Value × Need)                   Track 2: Steps 2.1-2.2
    │                                            ↑
    │ (sequential: Track 1 first)                │
    └─────── segments as moderators ─────────────┘
```

**Key Dependencies:**
- **Step 1.1 → Step 1.2**: Use NMF factors if useful, otherwise independent analysis
- **Track 1 → Track 2**: Sequential execution (complete Track 1 before Track 2)
- **Cross-Track**: Track 1 segments serve as HTE moderators in Track 2

### 2.2 Practical Usage Comparison

| Aspect | Track 1 (Descriptive) | Track 2 (Causal) |
|--------|----------------------|------------------|
| **Core Question** | "Who is this customer?" | "Will this intervention work for them?" |
| **Primary Users** | Marketing, CRM, Strategy | Data Science, Optimization |
| **Usage Timing** | Campaign planning, Strategy | Campaign execution, A/B optimization |
| **Difficulty** | Low | High |
| **Explainability** | "Premium Fresh Lover segment" | "CATE = 0.15 for this customer" |
| **Usage Frequency** | High (daily operations) | Medium (specific scenarios) |
| **Org Requirements** | Basic analytics capability | Causal thinking culture required |

### 2.3 Marketer's Perspective

**Track 1 (Almost certainly utilized):**
- Intuitive: "High-Value + Deal Seeker" → immediately understandable
- Easy to act on: Select segment → Design offer
- Easy to communicate: Explainable to executives and other departments
- Stable: Segments don't change frequently

**Track 2 (Depends on organizational maturity):**
- Requires conceptual understanding: "Treatment effect", "Uplift"
- Requires experimentation infrastructure: A/B testing systems
- Interpretation gap: Numbers need translation to actionable insights
- High value: Enables ROI optimization when properly implemented

---

## 3. Track 1: Customer Understanding

### 3.1 Methods

#### Step 1.1: Exploratory Analysis

**Objective:** Discover latent customer dimensions and derive base segments.

**Method:**
1. **Non-negative Matrix Factorization (NMF)**
   - Input: All 33 customer features
   - Output: Latent factor scores and loadings
   - Interpretation: Name factors based on loading patterns

2. **Clustering**
   - Algorithm: K-Means or GMM
   - Input: NMF factor scores (or raw features)
   - Output: Base customer segments

**Research Questions:**
- What are the latent dimensions explaining customer behavior?
- What customer types exist in the data?

#### Step 1.2: Value × Need Integration

**Objective:** Create actionable segments combining customer value and needs.

**Value Layer (RFM + Engagement):**
- Core features: recency, frequency, monetary_sales
- Extended: monetary_avg_basket_sales, purchase_regularity, week_coverage
- CLV comparison: Historical, RFM-based, Predictive (BG/NBD, Gamma-Gamma)
- Engagement Score: Composite of recency + frequency + regularity

**Need Layer (Behavioral + Category):**
- Behavioral: discount_rate, discount_usage_pct, private_label_ratio, n_departments, n_products, avg_items_per_basket, avg_products_per_basket
- Category: share_grocery, share_fresh, share_bakery, share_health_beauty, share_alcohol, share_other

**Integration:**
- Option A: Value-Need Matrix (Value tier × Need cluster)
- Option B: Sequential segmentation (Value tier → Need cluster within tier)

**Output:** "To whom (Value) should we offer what (Need)?"

### 3.2 Operational Definitions

| Item | Definition |
|------|------------|
| **Analysis Unit** | `household_key` (customer) |
| **Cohort** | Customers with at least 1 purchase in Week 1-102 (~2,500) |
| **Observation Period** | Week 1-102 (Full 102 weeks) |
| **Features** | 33 base features (RFM, Behavioral, Category, Time) |
| **Output** | Customer segments, Factor scores |

### 3.3 Features

#### Base Features (33)

| Group | Count | Features |
|-------|-------|----------|
| **Recency** | 6 | recency, recency_weeks, active_last_4w, active_last_12w, days_between_purchases_avg, days_between_purchases_std |
| **Frequency** | 6 | frequency, frequency_per_week, frequency_per_month, transaction_count, weeks_with_purchase, purchase_regularity |
| **Monetary** | 7 | monetary_sales, monetary_actual, monetary_avg_basket_sales, monetary_avg_basket_actual, monetary_std, monetary_per_week, coupon_savings_ratio |
| **Behavioral** | 7 | discount_rate, discount_usage_pct, private_label_ratio, n_departments, n_products, avg_items_per_basket, avg_products_per_basket |
| **Category** | 6 | share_grocery, share_fresh, share_bakery, share_health_beauty, share_alcohol, share_other |
| **Time** | 1 | week_coverage |

#### Feature Allocation

| Step | Feature Scope | Count |
|------|--------------|-------|
| **Step 1.1 Exploratory** | All 33 features → NMF | 33 |
| **Step 1.2a Value** | RFM + Engagement | ~21 |
| **Step 1.2b Need** | Behavioral + Category | 13 |

### 3.4 Evaluation

**Technical Metrics:**
- NMF: Reconstruction error, Sparsity
- Clustering: Silhouette Score, Calinski-Harabasz Index, Davies-Bouldin Index

**Interpretability:**
- Factor loadings interpretability
- Cluster separation (ANOVA, Chi-square tests)

**Business:**
- CLV difference significance across segments
- Actionability of segment definitions

**Stability:**
- Segment consistency across periods (Adjusted Rand Index)
- Customer segment migration rate

---

## 4. Track 2: Causal Targeting

### 4.1 Methods

#### Step 2.1: HTE Analysis

**Objective:** Estimate heterogeneous treatment effects of campaigns.

**Methods:**
- Meta-learners: S-Learner, T-Learner, X-Learner, R-Learner
- Tree-based: Causal Forest (GRF) - enables rule extraction

**Treatment Definition:**
- Campaign targeting (TypeA: targeted, TypeB/C: uniform)
- Coupon issuance

**Outcome Variables:**
- Coupon redemption
- Purchase amount increase
- Category purchase

**Covariates:**
- Track 1 features (RFM, Behavioral, Category)
- Track 1 segments (as moderators)
- Demographics (hh_demographic)
- Marketing exposure (display, mailer)

**Analysis Questions:**
1. Does treatment effect heterogeneity exist? (BLP test)
2. Which customer characteristics drive effect differences?
3. What is the CATE distribution?

#### Step 2.2: Optimal Policy

**Objective:** Derive optimal targeting rules and estimate policy value.

**Methods:**
- Subgroup discovery: Causal Tree rules
- Policy learning: Optimal treatment assignment based on CATE
- Policy value estimation: Expected outcome under policy

**Output:**
- Targeting rules (interpretable decision rules)
- Policy value (expected improvement vs. random targeting)
- ROI estimates under optimal policy

### 4.2 Operational Definitions (3 Scenarios)

**Campaign Data Overview:**

| Type | # Campaigns | # Targeting Records | Avg per Campaign |
|------|-------------|---------------------|------------------|
| TypeA | 5 | 3,979 | ~796 |
| TypeB | 19 | 2,655 | ~140 |
| TypeC | 6 | 574 | ~96 |

---

#### Scenario 1: TypeA Only (Primary)

| Item | Definition |
|------|------------|
| **Analysis Unit** | `household_key × CAMPAIGN` (TypeA campaigns only) |
| **Cohort** | Pre-treatment purchase history + Active during TypeA campaign period |
| **Treatment** | `targeted_typeA` (binary): Exists in TypeA campaign_table |
| **Control** | Active during TypeA campaign but not targeted |
| **Sample** | ~3,979 treatment + control |
| **Advantage** | Cleanest causal inference (Direct Mail = clear intervention) |

---

#### Scenario 2: All Types with Moderator

| Item | Definition |
|------|------------|
| **Analysis Unit** | `household_key × CAMPAIGN` (all 30 campaigns) |
| **Cohort** | Pre-treatment purchase history + Active during campaign period |
| **Treatment** | `campaign_targeted` (binary) |
| **Moderator** | `campaign_type` (A/B/C) |
| **Control** | Not targeted for that specific campaign |
| **Sample** | ~7,208 treatment + control |
| **Clustering** | Clustered SE by `household_key` |
| **Advantage** | Compare effects across campaign types |

---

#### Scenario 3: Customer-Level Aggregation

| Item | Definition |
|------|------------|
| **Analysis Unit** | `household_key` (customer) |
| **Cohort** | Pre-treatment purchase history + Active during campaign period |
| **Treatment Options** | `ever_targeted_typeA` (binary) or `n_typeA_campaigns` (count) |
| **Control** | Never targeted by TypeA campaigns |
| **Sample** | ~2,500 customers |
| **Advantage** | Simple, independent observations, overall effect |

---

#### Common Definitions (All Scenarios)

| Item | Definition |
|------|------------|
| **Outcome** | `redemption` (coupon usage), `purchase_amount` (campaign period) |
| **Confounders** | Pre-treatment Track 1 features (Week 1-31), Marketing exposure, Demographics |
| **Pre-treatment Period** | Week 1-31 (feature construction) |
| **Treatment Period** | Week 32-102 (campaign period) |

#### Scenario Selection Guide

| Research Question | Recommended Scenario |
|-------------------|---------------------|
| "What is the causal effect of TypeA campaigns?" | Scenario 1 |
| "How do effects differ across campaign types?" | Scenario 2 |
| "What is the overall marketing effect?" | Scenario 3 |
| "Which customers respond to TypeA?" (HTE) | Scenario 1 or 2 |

### 4.3 Causal Graph

#### DAG Structure

```
                    Marketing Exposure (display, mailer)
                           │
                           ↓
Campaign Targeting ──────→ Purchase ←────── Customer Characteristics
         │                                     (confounders)
         │                                          │
         └──────→ Coupon Redemption ←───────────────┘
```

#### Variable Roles

| Role | Variables |
|------|----------|
| **Treatment** | Campaign targeting (TypeA), Coupon issuance |
| **Confounders** | Marketing exposure (display, mailer), Customer characteristics (demographics, historical behavior) |
| **Outcome** | Coupon redemption, Purchase amount, Category purchase |
| **Effect Modifiers** | All covariates (for HTE analysis) |

#### Identification Strategy

**Assumptions:**
1. **Unconfoundedness**: Conditional on observed covariates, treatment assignment is independent of potential outcomes
2. **Overlap**: All customers have positive probability of treatment

**Debiasing Approach:**
- Include marketing exposure (display, mailer) as confounders
- Control for pre-treatment customer characteristics
- Use pre-campaign period (Week 1-31) features to avoid endogeneity

### 4.4 Features

**Treatment Variables:**
- `campaign_targeted`: Campaign targeting indicator (binary)
- `campaign_type`: Campaign type (A/B/C)

**Confounders - Marketing Exposure (from causal_data):**
- `display_exposure_rate`: Proportion of purchases with display exposure
- `mailer_exposure_rate`: Proportion of purchases with mailer exposure
- `mailer_type_A/B/C`: Exposure to each mailer type

**Confounders - Demographics (from hh_demographic):**
- AGE_GRP, INCOME_GRP, HOUSEHOLD_SIZE
- KID_CATEGORY_DESC, MARITAL_STATUS
- CLASSIFICATION_1~7

**Outcome Variables:**
- `redemption`: Coupon redemption indicator
- `purchase_amount`: Purchase amount in campaign period
- `category_purchase`: Target category purchase indicator

### 4.5 Evaluation

**Causal Metrics:**
- ATE significance (bootstrap CI, p-value)
- HTE existence test (BLP test, calibration)
- CATE distribution analysis

**Prediction Metrics:**
- Uplift curves (AUUC, Qini coefficient)
- Calibration plots

**Policy Metrics:**
- Policy value (expected outcome under policy)
- Targeting efficiency (precision@k)
- ROI under optimal policy vs. random targeting

**Robustness & Validation:**
- Bootstrap variance
- Cross-validation stability
- Model comparison (CATE correlation across different models)

**Causal Validation (Ground truth unavailable):**
- **Refutation tests:**
  - Placebo treatment: Random treatment → effect ≈ 0
  - Random confounder: Add random variable → effect unchanged
  - Subset validation: Similar results on data subsets
- **Sensitivity analysis:**
  - E-value: How strong would unmeasured confounding need to be?
  - Rosenbaum bounds: Sensitivity to hidden bias
- **Calibration:**
  - RATE (Sorted Group ATE): Do high-CATE groups have higher actual effects?

---

## 5. Period & Validation Design

### 5.1 Data Availability

| Data | Period | Notes |
|------|--------|-------|
| Transactions | Day 1-711 (Week 1-102) | Full 2 years |
| Campaigns | Day 224-719 (Week 32-102) | Campaigns only in later period |
| causal_data | Week 9-101 | Marketing exposure data |

**Key Periods:**
- Week 1-31: Pre-campaign period (no campaigns)
- Week 32-102: Campaign period

### 5.2 Track 1 Period Design

| Setting | Value | Rationale |
|---------|-------|----------|
| **Analysis Period** | Week 1-102 (Full) | Maximum data, complete customer behavior |
| **Stability Check** | Compare half-periods | Validate segment consistency over time |

### 5.3 Track 2 Period Design

| Setting | Value | Rationale |
|---------|-------|----------|
| **Analysis Period** | Week 32-102 | Campaign period only (treatment data available) |
| **Pre-treatment Features** | Week 1-31 | Use pre-campaign behavior as confounders (avoid information leakage) |
| **Validation** | TBD | Options: Time-based split, Campaign-based split, Cross-validation |

### 5.4 Customer Coverage

| Period | Weeks | Customers | Coverage |
|--------|-------|-----------|----------|
| Full (1-102) | 102 | 2,500 | 100% |
| Campaign (32-102) | 71 | 2,490 | 99.6% |
| Last 52w (51-102) | 52 | 2,479 | 99.2% |

---

## 6. Implementation Plan

### 6.1 Notebooks

| Notebook | Track | Content |
|----------|-------|--------|
| `00_study_design.ipynb` | - | Study design documentation (this notebook) |
| `01_feature_engineering.ipynb` | 1 & 2 | Build all customer features |
| `02_exploratory_segments.ipynb` | 1.1 | NMF + Clustering |
| `03_clv_behavior_segments.ipynb` | 1.2 | CLV × Behavior segmentation |
| `04_hte.ipynb` | 2.1 | HTE analysis |
| `05_optimal_policy.ipynb` | 2.2 | Optimal policy learning |

### 6.2 Implementation Order

```
Track 1 (Priority: High)
────────────────────────
1. Feature Engineering (01_feature_engineering.ipynb)
   - Build all 33 base features
   - Implement Value and Need feature groups

2. Step 1.1 Exploratory (02_exploratory_segments.ipynb)
   - NMF factor analysis
   - Base clustering

3. Step 1.2 CLV × Behavior (03_clv_behavior_segments.ipynb)
   - CLV comparison
   - Value-Need integration

Track 2 (Priority: Medium)
────────────────────────
4. Step 2.1 HTE Analysis (04_hte.ipynb)
   - Build campaign features
   - CATE estimation
   - Heterogeneity testing

5. Step 2.2 Optimal Policy (05_optimal_policy.ipynb)
   - Subgroup discovery
   - Policy evaluation
```

---

## 7. Cohort Definition & Comparison

This section loads data, builds cohorts using `src/cohorts.py`, and compares them.

In [7]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

from data.dunnhumby.eda.exploration_2 import DATA_PATH
from projects.segmentation_dunnhumby.src.cohorts import (
    define_track1_cohort,
    define_track2_base_cohort,
    build_scenario1_cohort,
    build_scenario2_cohort,
    build_scenario3_cohort,
    compare_cohorts
)

PosixPath('/Users/taehyun/project/side-projects/data')

In [22]:
DATA_PATH

PosixPath('/Users/taehyun/project/side-projects/data/dunnhumbmy/raw')

In [23]:
# Load data
DATA_PATH = Path('.').absolute().parents[2] / 'data' / 'dunnhumby' / 'raw'

df_trans = pd.read_csv(DATA_PATH / 'transaction_data.csv')
df_campaign_table = pd.read_csv(DATA_PATH / 'campaign_table.csv')
df_campaign_desc = pd.read_csv(DATA_PATH / 'campaign_desc.csv')

print(f"Transactions: {len(df_trans):,} rows")
print(f"Campaign targeting: {len(df_campaign_table):,} rows")
print(f"Campaigns: {len(df_campaign_desc)} campaigns")

Transactions: 2,595,732 rows
Campaign targeting: 7,208 rows
Campaigns: 30 campaigns


In [24]:
# Build Track 1 Cohort
track1_cohort = define_track1_cohort(df_trans)
print(f"Track 1 Cohort: {len(track1_cohort):,} customers")

Track 1 Cohort: 2,500 customers


In [25]:
# Build Track 2 Base Cohort
track2_base = define_track2_base_cohort(df_trans)
print(f"Track 2 Base Cohort: {len(track2_base):,} customers")
print(f"  (requires purchase in Week 1-31 AND Week 32-102)")

Track 2 Base Cohort: 2,484 customers
  (requires purchase in Week 1-31 AND Week 32-102)


In [26]:
# Build Scenario 1: TypeA Only
scenario1 = build_scenario1_cohort(df_trans, df_campaign_table, df_campaign_desc)
print(f"Scenario 1 (TypeA Only):")
print(f"  Total rows: {len(scenario1):,}")
print(f"  Customers: {scenario1['household_key'].nunique():,}")
print(f"  Treatment: {scenario1['targeted'].sum():,} ({scenario1['targeted'].mean()*100:.1f}%)")
print(f"  Control: {(1-scenario1['targeted']).sum():,}")

Scenario 1 (TypeA Only):
  Total rows: 12,420
  Customers: 2,484
  Treatment: 3,978 (32.0%)
  Control: 8,442


In [27]:
# Build Scenario 2: All Types with Moderator
scenario2 = build_scenario2_cohort(df_trans, df_campaign_table, df_campaign_desc)
print(f"Scenario 2 (All Types):")
print(f"  Total rows: {len(scenario2):,}")
print(f"  Customers: {scenario2['household_key'].nunique():,}")
print(f"  Treatment: {scenario2['targeted'].sum():,} ({scenario2['targeted'].mean()*100:.1f}%)")
print(f"\n  By campaign type:")
print(scenario2.groupby('campaign_type')['targeted'].agg(['sum', 'mean']).round(3))

Scenario 2 (All Types):
  Total rows: 74,520
  Customers: 2,484
  Treatment: 7,206 (9.7%)

  By campaign type:
                sum   mean
campaign_type             
TypeA          3978  0.320
TypeB          2654  0.056
TypeC           574  0.039


In [28]:
# Build Scenario 3: Customer-Level
scenario3 = build_scenario3_cohort(df_trans, df_campaign_table, df_campaign_desc)
print(f"Scenario 3 (Customer-Level):")
print(f"  Customers: {len(scenario3):,}")
print(f"  Ever targeted (TypeA): {scenario3['ever_targeted_typeA'].sum():,} ({scenario3['ever_targeted_typeA'].mean()*100:.1f}%)")
print(f"  Ever targeted (Any): {scenario3['ever_targeted_any'].sum():,} ({scenario3['ever_targeted_any'].mean()*100:.1f}%)")
print(f"\n  TypeA campaigns per customer:")
print(scenario3['n_typeA_campaigns'].value_counts().sort_index())

Scenario 3 (Customer-Level):
  Customers: 2,484
  Ever targeted (TypeA): 1,512 (60.9%)
  Ever targeted (Any): 1,583 (63.7%)

  TypeA campaigns per customer:
n_typeA_campaigns
0    972
1    325
2    326
3    580
4    144
5    137
Name: count, dtype: int64


In [29]:
# Cohort Comparison Summary
print("=" * 60)
print("COHORT COMPARISON SUMMARY")
print("=" * 60)

comparison = pd.DataFrame([
    {
        'Cohort': 'Track 1',
        'Unit': 'Customer',
        'N_Rows': len(track1_cohort),
        'N_Customers': len(track1_cohort),
        'N_Treatment': '-',
        'Treatment_%': '-'
    },
    {
        'Cohort': 'Track 2 Base',
        'Unit': 'Customer',
        'N_Rows': len(track2_base),
        'N_Customers': len(track2_base),
        'N_Treatment': '-',
        'Treatment_%': '-'
    },
    {
        'Cohort': 'Scenario 1 (TypeA)',
        'Unit': 'Customer × Campaign',
        'N_Rows': len(scenario1),
        'N_Customers': scenario1['household_key'].nunique(),
        'N_Treatment': scenario1['targeted'].sum(),
        'Treatment_%': f"{scenario1['targeted'].mean()*100:.1f}%"
    },
    {
        'Cohort': 'Scenario 2 (All)',
        'Unit': 'Customer × Campaign',
        'N_Rows': len(scenario2),
        'N_Customers': scenario2['household_key'].nunique(),
        'N_Treatment': scenario2['targeted'].sum(),
        'Treatment_%': f"{scenario2['targeted'].mean()*100:.1f}%"
    },
    {
        'Cohort': 'Scenario 3 (Customer)',
        'Unit': 'Customer',
        'N_Rows': len(scenario3),
        'N_Customers': len(scenario3),
        'N_Treatment': scenario3['ever_targeted_typeA'].sum(),
        'Treatment_%': f"{scenario3['ever_targeted_typeA'].mean()*100:.1f}%"
    }
])

comparison

COHORT COMPARISON SUMMARY


,Cohort,Unit,N_Rows,N_Customers,N_Treatment,Treatment_%
0,Track 1,Customer,2500,2500,-,-
1,Track 2 Base,Customer,2484,2484,-,-
2,Scenario 1 (TypeA),Customer × Campaign,12420,2484,3978,32.0%
3,Scenario 2 (All),Customer × Campaign,74520,2484,7206,9.7%
4,Scenario 3 (Customer),Customer,2484,2484,1512,60.9%
